<a href="https://colab.research.google.com/github/nanpolend/machine-learning/blob/master/kaggle%E9%90%B5%E9%81%94%E5%B0%BC%E8%99%9F%E7%94%9F%E5%AD%98%E9%A0%90%E6%B8%AC_deepseek%E4%BF%AE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 安裝必要套件
!pip install pandas scikit-learn

In [8]:
"""
🏆 鐵達尼號生存預測系統完整修正版

     +---------------------+
     |   資料載入模組        |
     |  (CSV檔案讀取)       |
     +----------+-----------+
                |
     +----------v-----------+
     | 資料預處理模組         |
     | (缺失值處理/特徵工程)   |
     +----------+-----------+
                |
     +----------v-----------+
     | 特徵工程模組          |
     | (數值轉換/特徵選擇)    |
     +----------+-----------+
                |
     +----------v-----------+
     | 模型訓練模組          |
     | (隨機森林分類器)      |
     +----------+-----------+
                |
     +----------v-----------+
     | 預測輸出模組          |
     | (生成提交檔案)        |
     +---------------------+
"""

# -*- coding: utf-8 -*-
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV

class 生存預測系統:
    def __init__(self):
        """初始化系統元件"""
        self.模型 = None
        self.訓練資料 = None
        self.測試資料 = None
        self.特徵列 = [
            'Pclass', 'Sex', 'Age',
            'SibSp', 'Parch', 'Fare',
            'Embarked', 'FamilySize',
            'Title', 'HasCabin'
        ]

    def 載入資料(self, 訓練路徑='train.csv', 測試路徑='test.csv'):
        """
        📂 安全資料載入功能
        修正重點：確保檔案存在性檢查
        """
        try:
            # 嚴格檔案檢查
            if not os.path.isfile(訓練路徑):
                raise FileNotFoundError(f"訓練檔案路徑不存在：{os.path.abspath(訓練路徑)}")
            if not os.path.isfile(測試路徑):
                raise FileNotFoundError(f"測試檔案路徑不存在：{os.path.abspath(測試路徑)}")

            # 讀取資料並驗證基本結構
            self.訓練資料 = pd.read_csv(訓練路徑)
            self.測試資料 = pd.read_csv(測試路徑)

            # 關鍵欄位驗證
            必要欄位 = {'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Cabin', 'Name'}
            for col in 必要欄位:
                if col not in self.訓練資料.columns:
                    raise ValueError(f"訓練集缺少必要欄位：{col}")
                if col not in self.測試資料.columns and col != 'Survived':
                    raise ValueError(f"測試集缺少必要欄位：{col}")

            print("✅ 資料載入成功")
            print(f"訓練集樣本數：{len(self.訓練資料)}")
            print(f"測試集樣本數：{len(self.測試資料)}")
            return True

        except Exception as 錯誤:
            print(f"❌ 資料載入失敗：{str(錯誤)}")
            print("💡 解決建議：")
            print("1. 確認檔案位於當前工作目錄")
            print(f"2. 當前工作目錄內容：{os.listdir()}")
            print("3. 確認檔案名稱完全匹配")
            return False

    def 預處理資料(self):
        """
        🔧 強化型資料預處理
        修正重點：統一使用原始英文欄位名稱
        """
        try:
            # 合併資料集
            合併資料 = pd.concat([self.訓練資料, self.測試資料], axis=0)

            # 缺失值處理（使用原始欄位名）
            合併資料['Age'].fillna(合併資料['Age'].median(), inplace=True)  # 年齡
            合併資料['Fare'].fillna(合併資料['Fare'].median(), inplace=True)  # 票價
            合併資料['Embarked'].fillna(合併資料['Embarked'].mode()[0], inplace=True)  # 登船港口

            # 特徵工程
            合併資料['FamilySize'] = 合併資料['SibSp'] + 合併資料['Parch'] + 1  # 家庭人數
            合併資料['HasCabin'] = 合併資料['Cabin'].notnull().astype(int)  # 有無艙室
            合併資料['Title'] = 合併資料['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)  # 稱謂提取

            # 稱謂分組標準化
            合併資料['Title'] = 合併資料['Title'].replace([
                'Lady', 'Countess','Capt', 'Col','Don',
                'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'
            ], 'Rare')
            合併資料['Title'] = 合併資料['Title'].replace(['Mlle','Ms'], 'Miss')
            合併資料['Title'] = 合併資料['Title'].replace('Mme', 'Mrs')

            # 類別變數編碼
            編碼器 = LabelEncoder()
            合併資料['Sex'] = 編碼器.fit_transform(合併資料['Sex'])  # 性別
            合併資料['Embarked'] = 編碼器.fit_transform(合併資料['Embarked'])  # 登船港口
            合併資料['Title'] = 編碼器.fit_transform(合併資料['Title'])  # 稱謂

            # 資料分割
            self.訓練資料 = 合併資料.iloc[:len(self.訓練資料)]
            self.測試資料 = 合併資料.iloc[len(self.訓練資料):].drop('Survived', axis=1, errors='ignore')

            print("✅ 資料預處理完成")
            print(f"有效特徵數量：{len(self.特徵列)}")

        except Exception as 錯誤:
            print(f"❌ 預處理失敗：{str(錯誤)}")
            raise

    def 訓練模型(self):
        """
        🤖 強化模型訓練流程
        新增功能：交叉驗證與參數調優
        """
        try:
            # 準備訓練數據
            X_train = self.訓練資料[self.特徵列]
            y_train = self.訓練資料['Survived']

            # 參數網格設定
            參數組合 = {
                'n_estimators': [100, 200, 300],
                'max_depth': [5, 10, 15],
                'min_samples_split': [2, 5, 10]
            }

            # 初始化模型
            self.模型 = GridSearchCV(
                estimator=RandomForestClassifier(random_state=42),
                param_grid=參數組合,
                cv=5,
                scoring='accuracy',
                verbose=1
            )

            # 訓練模型
            print("🔄 開始模型訓練...")
            self.模型.fit(X_train, y_train)

            # 輸出結果
            print(f"🎯 最佳參數組合：{self.模型.best_params_}")
            print(f"🏅 交叉驗證最佳準確率：{self.模型.best_score_:.2%}")

        except Exception as 錯誤:
            print(f"❌ 模型訓練失敗：{str(錯誤)}")
            raise

    def 生成預測(self, 輸出路徑='submission.csv'):
        """
        📤 安全預測生成功能
        新增功能：結果驗證
        """
        try:
            # 檢查模型狀態
            if self.模型 is None:
                raise ValueError("尚未訓練模型！")

            # 生成預測
            X_test = self.測試資料[self.特徵列]
            預測結果 = self.模型.predict(X_test)

            # 結果驗證
            if len(預測結果) != len(self.測試資料):
                raise ValueError("預測結果數量不匹配！")

            # 建立提交檔案
            提交資料 = pd.DataFrame({
                'PassengerId': self.測試資料['PassengerId'],
                'Survived': 預測結果.astype(int)
            })

            # 保存檔案
            提交資料.to_csv(輸出路徑, index=False)
            print(f"✅ 提交檔案已保存至：{os.path.abspath(輸出路徑)}")
            print("📊 預測結果分佈：")
            print(提交資料['Survived'].value_counts())

        except Exception as 錯誤:
            print(f"❌ 預測生成失敗：{str(錯誤)}")
            raise

if __name__ == "__main__":
    # 初始化系統
    預測系統 = 生存預測系統()

    # 執行完整流程
    if 預測系統.載入資料():
        try:
            預測系統.預處理資料()
            預測系統.訓練模型()
            預測系統.生成預測()
        except Exception as 錯誤:
            print(f"🔴 系統執行中斷：{str(錯誤)}")
            print("建議檢查步驟：")
            print("1. 數據預處理是否正確")
            print("2. 特徵列定義是否匹配")
            print("3. 模型參數是否合理")

✅ 資料載入成功
訓練集樣本數：891
測試集樣本數：418
✅ 資料預處理完成
有效特徵數量：10
🔄 開始模型訓練...
Fitting 5 folds for each of 27 candidates, totalling 135 fits


<ipython-input-8-7b9badd0b6cb>:98: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  合併資料['Age'].fillna(合併資料['Age'].median(), inplace=True)  # 年齡
<ipython-input-8-7b9badd0b6cb>:99: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True

🎯 最佳參數組合：{'max_depth': 10, 'min_samples_split': 10, 'n_estimators': 100}
🏅 交叉驗證最佳準確率：83.28%
✅ 提交檔案已保存至：/content/submission.csv
📊 預測結果分佈：
Survived
0    265
1    153
Name: count, dtype: int64
